### Notebook for data load and EDA

In [1]:
# libs import
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
import lux
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'pandas'